In [ ]:
# Ideas for linear regression. 
# Time vs. Collision Rates
# Multiple regressions on one plot based on mode or county.
# Mileage vs. Collision Rates
# Mutliple regressions on one based on injury outcome.
# Time vs. Injury Total Per Collision